<a href="https://colab.research.google.com/github/albim72/ML_ZAAWANSOWANY_11/blob/main/CNN_obiektowo_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Conv2D
from tensorflow.keras import Model

In [11]:
mnist = tf.keras.datasets.mnist

(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = x_train/255.0,x_test/255.0

#dodanie kanałow wymiarów
x_train = x_train[...,tf.newaxis].astype("float32")
x_test = x_test[...,tf.newaxis].astype("float32")

In [12]:
#deklaracja przetwarzania wsadowego i przetasowanie zestawu danych
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train,y_train)
).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [13]:
class MyModel(Model):
  def __init__(self) -> None:
    super(MyModel,self).__init__()
    self.conv1 = Conv2D(32,3,activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128,activation='relu')
    self.d2 = Dense(10)

  def __call__(self,x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [14]:
#wybór optymalizatora i funkcji strat
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [15]:
#metryki do pomiaru dokładności i straty
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [17]:
@tf.function
def train_step(images,labels):
  with tf.GradientTape() as tape:
    predictions = model(images,training = True)
    loss = loss_object(labels,predictions)
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels,predictions)

In [18]:
@tf.function
def test_step(images,labels):
  predictions = model(images,training = False)
  t_loss = loss_object(labels,predictions)

  test_loss(t_loss)
  test_accuracy(labels,predictions)